In [1]:
import numpy as np
import os

# Convert BVH to CSV Files

Hacky code to pull out just the local acceleration data across joints / frames

In [2]:
#Don't need to rerun - used to create dsv_data
def convert_file(file_in, file_out):
    file = open(file_in, "r")
    full_data = file.readlines()
    i = full_data.index("MOTION\n")
    frames = int(full_data[i+1].strip().replace("Frames: ", ""))
    frame_time = float(full_data[i+2].strip().replace("Frame Time: ", ""))
    # print(frames, frame_time)

    pos_data = [] # frame [ 83 joints [xpos, ypos, zpos, zrot, xrot, yrot]]]
    for frame in full_data[i+3: ]:
        pos_data.append( [float(x) for x in frame.split()])
    
    # take the derivative to get acceleration
    pos_data = np.array(pos_data)
    v_data = np.diff(pos_data, 1, 0)
    acc_data = np.diff(pos_data, 2, 0)
    
    # How much does taking the derivative reduce the amount of non zero data?
    total_size = np.size(pos_data)
    # print(total_size , round(np.count_nonzero(pos_data)/total_size*100,4), round(np.count_nonzero(acc_data)/total_size*100, 4))
    # ~ 70 -> 20%
    
    # cut out the first ~5 seconds on each side of the take to remove weird artifacts
    acc_data = acc_data[500: -500]
    
    np.savetxt(file_out, acc_data, delimiter=',', fmt='%d')
    # TODO - is there a better way to store this that capitalizes on the sparsity?

    file.close()
    return len(acc_data), frame_time

Go through each pair of bvh files and convert to csv with new naming convention

In [4]:
#Don't need to rerun - used to create dsv_data
data_dir = "mocap_data"

frames = []
frame_length = []

# iterate through the session directories
for session_dir in os.listdir(data_dir):
    if not session_dir.startswith("session"):
        continue
    session_path = os.path.join(data_dir, session_dir)
    for take_dir in os.listdir(session_path):
        if not take_dir.startswith("take"):
            continue
        take_path = os.path.join(session_path, take_dir)
        file_a, file_b =  [x for x in os.listdir(take_path) if ".bvh" in x]
        person_a = file_a.replace(take_dir + "_hasFingers_", "").replace("_scale_local.bvh", "")
        person_b = file_b.replace(take_dir + "_hasFingers_", "").replace("_scale_local.bvh", "")
        new_a = "csv_data/" + person_a + "_" + person_b + "_" + session_dir.replace("ession", "") + take_dir.replace("ake", "") + '.csv'
        new_b = "csv_data/" + person_b + "_" + person_a + "_" + session_dir.replace("ession", "") + take_dir.replace("ake", "") + '.csv'
        f, f_len = convert_file(os.path.join(take_path, file_a), new_a)
        frames.append(f)
        frame_length.append(f_len)
        f, f_len
        convert_file(os.path.join(take_path, file_b), new_b)
        frames.append(f)
        frame_length.append(f_len)
        
print(frames[0])

# Do we need to chop up any of our data?
print(np.min(frames), np.max(frames)/np.min(frames)) # 5226 min length of motion
      
# Are all frames captured at the same frame rate?
print(frame_length[0], len([x for x in frame_length if x!= frame_length[0]]))

# Load in CSV to create data matrix

In [28]:
data = None
l = 5226 #56 files
speaker = []
audience =  []

# format data so that each column is a  // WHY IS THIS SOOOOO SLOW?
for file in [x for x in os.listdir("csv_data") if ".csv" in x]:
    print(file)
    d = np.loadtxt(os.path.join("csv_data", file), delimiter=',')
    for i in range(d.shape[0]//l): # use numpy reshape instead??
        chunk = np.array(d[i*l: (i+1)*l]).flatten()
        speaker.append(info[0])
        audience.append(info[1])
        if (data is None):
            data = [chunk]
        else:
            data.append(chunk)
            
data = np.transpose(np.array(data)) # each column is a series of accelerations from within a take

In [35]:
print(data.shape)
np.savetxt("matrix_data.csv", data, delimiter=',', fmt='%d')
# TODO: This is a sparse matrix so we can store this even more efficiently - scipy might do this

(2602548, 526)


In [30]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 14.1 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.2 MB/s eta 0:00:00


In [32]:
import pandas as pd
from sklearn.decomposition import PCA

## Is there any other preprocessing that we need to do - normalization or otherwise?

## TODO, SVD -> PCA, Or direct PCA to reduce dimensionality

In [ ]:
# reduce down to subset of features
pca = PCA(n_components=2)
pca_feats = pca.fit_transform(data)

In [ ]:
# NOt working yet PCA fit takes too long. Code from demo  - https://www.jcchouinard.com/pca-with-python/#PCA_Examples_From_This_Tutorial

# Create dataframe
pca_df = pd.DataFrame(
    data=pca_features, 
    columns=['PC1', 'PC2'])
 
# map target names to PCA features   
target_names = {
    0:'setosa',
    1:'versicolor', 
    2:'virginica'
}
 
pca_df['target'] = y
pca_df['target'] = pca_df['target'].map(target_names)
 
pca_df.head()

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
 
sns.lmplot(
    x='PC1', 
    y='PC2', 
    data=pca_df, 
    hue='target', 
    fit_reg=False, 
    legend=True
    )
 
plt.title('2D PCA Graph')
plt.show()